# Pytorch常用模块
在训练神经网络过程中，需要用到很多工具，其中最重要的三部分是：
- 数据
- 可视化
- `GPU`加速

本章主要介绍Pytorch在这几方面的工具模块，合理使用这些工具能够极大地提高编码效率。

## 数据处理

### 数据加载

在`PyTorch`中，数据加载可通过自定义的数据集对象。数据集对象被抽象为`Dataset`类，实现自定义的数据集需要继承`Dataset`，并实现两个`Python`魔法方法：
- `__getitem__`：返回一条数据，或一个样本。`obj[index]`等价于`obj.__getitem__(index)`
- `__len__`：返回样本的数量。`len(obj)`等价于`obj.__len__()`

以`Kaggle`经典挑战赛["Dogs vs. Cat"](https://www.kaggle.com/c/dogs-vs-cats/)的数据为例，来详细讲解如何处理数据。`"Dogs vs. Cats"`是一个分类问题，判断一张图片是狗还是猫，其所有图片都存放在一个文件夹下，根据文件名的前缀判断是狗还是猫。

In [2]:
%env LS_COLORS = None 
!tree --charset ascii  data/dogcat/

env: LS_COLORS=None
参数太多 - ascii


In [3]:
import torch as t
from torch.utils import data

In [4]:
import os
from PIL import  Image
import numpy as np

class DogCat(data.Dataset):
    def __init__(self, root):
        imgs = os.listdir(root)
        # 所有图片的绝对路径
        # 这里不实际加载图片，只是指定路径，当调用__getitem__时才会真正读图片
        self.imgs = [os.path.join(root, img) for img in imgs]
        
    def __getitem__(self, index):
        img_path = self.imgs[index]
        # dog->1， cat->0
        label = 1 if 'dog' in img_path.split('/')[-1] else 0
        pil_img = Image.open(img_path)
        array = np.asarray(pil_img)
        data = t.from_numpy(array)
        return data, label
    
    def __len__(self):
        return len(self.imgs)

In [5]:
dataset = DogCat('./data/dogcat/')
img, label = dataset[0] # 相当于调用dataset.__getitem__(0)
for img, label in dataset:
    print(img.size(), img.float().mean(), label)

torch.Size([500, 497, 3]) tensor(106.4915) 0
torch.Size([499, 379, 3]) tensor(171.8085) 0
torch.Size([236, 289, 3]) tensor(130.3004) 0
torch.Size([374, 499, 3]) tensor(115.5177) 0
torch.Size([375, 499, 3]) tensor(116.8138) 1
torch.Size([375, 499, 3]) tensor(150.5079) 1
torch.Size([377, 499, 3]) tensor(151.7174) 1
torch.Size([400, 300, 3]) tensor(128.1550) 1


#### Note
上面的代码自定义了自己的数据集，并可以依次获取。但这里返回的数据不适合实际使用，因其具有如下两方面问题：
- 返回样本的形状不一，因每张图片的大小不一样，这对于需要取`batch`训练的神经网络来说很不友好
- 返回样本的数值较大，未归一化至`[-1, 1]`

针对上述问题，`PyTorch`提供了`torchvision`[^1]。它是一个视觉工具包，提供了很多视觉图像处理的工具，其中`transforms`模块提供了对`PIL Image`对象和`Tensor`对象的常用操作。

> 对`PIL Image`的操作包括：
> - `Scale`：调整图片尺寸，长宽比保持不变
> - `CenterCrop`、`RandomCrop`、`RandomResizedCrop`：裁剪图片
> - `Pad`：填充
> - `ToTensor`：将`PIL Image`对象转成`Tensor`，会自动将`[0, 255]`归一化至`[0, 1]`

> 对`Tensor`的操作包括：
> - `Normalize`：标准化，即减均值，除以标准差
> - `ToPILImage`：将`Tensor`转为`PIL Image`对象

如果要对图片进行多个操作，可通过`Compose`函数将这些操作拼接起来，类似于`nn.Sequential`。注意，这些操作定义后是以函数的形式存在，真正使用时需调用它的`__call__`方法，这点类似于`nn.Module`。
> 例如要将图片调整为`224×224`，首先应构建这个操作`trans = Resize((224, 224))`，然后调用`trans(img)`。下面我们就用`transforms`的这些操作来优化上面实现的`dataset`。
[^1]: https://github.com/pytorch/vision/

In [6]:
import os
from PIL import  Image
import numpy as np
from torchvision import transforms as T

transform = T.Compose([
    T.Resize(224), # 缩放图片(Image)，保持长宽比不变，最短边为224像素
    T.CenterCrop(224), # 从图片中间切出224*224的图片
    T.ToTensor(), # 将图片(Image)转成Tensor，归一化至[0, 1]
    T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]) # 标准化至[-1, 1]，规定均值和标准差
])

class DogCat(data.Dataset):
    def __init__(self, root, transforms=None):
        imgs = os.listdir(root)
        self.imgs = [os.path.join(root, img) for img in imgs]
        self.transforms=transforms
        
    def __getitem__(self, index):
        img_path = self.imgs[index]
        label = 0 if 'dog' in img_path.split('/')[-1] else 1
        data = Image.open(img_path)
        if self.transforms:
            data = self.transforms(data)
        return data, label
    
    def __len__(self):
        return len(self.imgs)

dataset = DogCat('./data/dogcat/', transforms=transform)
img, label = dataset[0]
for img, label in dataset:
    print(img.size(), label)

torch.Size([3, 224, 224]) 1
torch.Size([3, 224, 224]) 1
torch.Size([3, 224, 224]) 1
torch.Size([3, 224, 224]) 1
torch.Size([3, 224, 224]) 0
torch.Size([3, 224, 224]) 0
torch.Size([3, 224, 224]) 0
torch.Size([3, 224, 224]) 0
